#                                                                           Aadhar Card detection using YOLO(YOU ONLY LOOK ONCE)

# Setup

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.


In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-185-g2334aa7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. Detect

`detect.py` runs YOLOv5 inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases), and saving results to `runs/detect`. Example inference sources are:

```shell
python detect.py --source 0  # webcam
                          img.jpg  # image
                          vid.mp4  # video
                          screen  # screenshot
                          path/  # directory
                         'path/*.jpg'  # glob
                         'https://youtu.be/Zgi9g1ksQHc'  # YouTube
                         'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream
```


In [ ]:
!python detect.py --weights runs/train/exp/weights/last.pt --img 608 --conf 0.25 --source ""# Replace with actual test data path
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

# 2. Finding Bounding box Coordinates

In [9]:
import torch
from PIL import Image
import os

# Load the custom trained YOLO model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='//content//yolov5//runs//train//exp//weights//best.pt')

# Folder paths for input images and output bounding box coordinates
input_folder = '//content//drive//MyDrive//Aadhar_dataset//test//images'
output_folder = '//content//bounding_box_coordinates'

# Perform object detection on images in the input folder
for filename in os.listdir(input_folder):
    # Check if the file has an image extension
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        # Construct the full file paths
        input_image_path = os.path.join(input_folder, filename)
        output_bb_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")

        # Load and preprocess the image
        image = Image.open(input_image_path)

        # Perform object detection using the custom YOLO model
        results = model(image)

        # Extract bounding box coordinates from the results
        bounding_boxes = results.xyxy[0][:, :4].tolist()

        # Save the bounding box coordinates to a text file
        with open(output_bb_path, 'w') as file:
            for box in bounding_boxes:
                x1, y1, x2, y2 = box
                file.write(f"{x1} {y1} {x2} {y2}\n")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 v7.0-185-g2334aa7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


# 3. Cropping Images

In [ ]:
import os
from PIL import Image

def crop_images_with_coordinates(input_folder, output_folder, detections_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get the list of image files in the input folder
    image_files = os.listdir(input_folder)

    for image_file in image_files:
        # Get the full path of the image
        image_path = os.path.join(input_folder, image_file)

        # Get the corresponding detections file path
        detections_file_name = os.path.splitext(image_file)[0] + '.txt'
        detections_file_path = os.path.join(detections_folder, detections_file_name)

        # Read the bounding box coordinates from the detections file
        with open(detections_file_path, 'r') as detections_file:
            coordinates = detections_file.readline().strip().split(' ')
            coordinates = list(map(float, coordinates))

        # Generate the output path for the cropped image
        output_path = os.path.join(output_folder, image_file)

        # Crop and save the image
        crop_image(image_path, coordinates, output_path)

def crop_image(image_path, coordinates, output_path):
    # Open the image
    image = Image.open(image_path)

    # Crop the image based on the coordinates
    cropped_image = image.crop(coordinates)

    # Save the cropped image
    cropped_image.save(output_path)

# Example usage
input_folder = "//content//yolov5//runs//detect//exp"
output_folder = "//content//cropped_images"
detections_folder = "//content//bounding_box_coordinates"

crop_images_with_coordinates(input_folder, output_folder, detections_folder)


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="C:\\Users\\sastr\\Pictures\\Screenshots\\detected 1.png" width="600">


# 4. Validate

Validate a model's accuracy on the [COCO](https://cocodataset.org/#home) dataset's `val` or `test` splits. Models are downloaded automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases). To show results by class use the `--verbose` flag.


In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

100%|██████████| 780M/780M [00:12<00:00, 66.6MB/s]


In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights yolov5s.pt --data coco.yaml --img 640 --half

val: data=/content/yolov5/data/coco.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-136-g71244ae Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
val: Scanning /content/datasets/coco/val2017... 4952 images, 48 backgrounds, 0 corrupt: 100% 5000/5000 [00:02<00:00, 2024.59it/s]
val: New cache created: /content/datasets/coco/val2017.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 157/157 [01:25<00:00,  1.84it/s]
                   all       5000      36335      0.671      0.519      0.566      0.371
Speed: 0.1ms pre-process, 3.1ms inference, 2.3ms NMS per image at shape (32,

In [ ]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

# 5. Training

In [3]:
# Train YOLOv5s on Custom_data for 30 epochs
!python train.py --img 608 --batch 8 --epochs 30 --data custom_data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=custom_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=8, imgsz=608, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-185-g2334aa7 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.

# 6. Appendix

Additional content below.


In [ ]:
# YOLOv5 PyTorch HUB Inference (DetectionModels only)
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', force_reload=True)  # yolov5n - yolov5x6 or custom
im = 'https://ultralytics.com/images/zidane.jpg'  # file, Path, PIL.Image, OpenCV, nparray, list
results = model(im)  # inference
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.